In [2]:
import torch
from torch import nn
from d2l import torch as d2l

batch_size = 64
train_iter, test_iter, vocab = d2l.load_data_imdb(batch_size)

In [10]:
for i in train_iter:
    print(len(i))
    print(len(i[0]))
    print(len(i[1]))
    print(i[0].shape)
    break

2
64
64
torch.Size([64, 500])


In [42]:
aa = next(iter(train_iter))

In [43]:
aa

[tensor([[   10,    38,   181,  ...,     0,     0,     0],
         [  122,   253,     5,  ...,     0,     0,     0],
         [  282,     8,  1534,  ...,     0,     0,     0],
         ...,
         [   48,     6,    31,  ...,     0,     0,     0],
         [  784,  1526,     6,  ...,     0,     0,     0],
         [ 5249, 35992,     0,  ...,     0,     0,     0]]),
 tensor([0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1,
         1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0])]

In [11]:
input = torch.randn(1,3,5)

In [12]:
input

tensor([[[ 1.1743,  0.8453, -1.2852,  0.1323, -0.3739],
         [ 0.0284,  0.7075,  0.4223,  0.4431, -0.2304],
         [ 0.2463,  0.8469, -0.7575, -2.4849, -2.0079]]])

In [24]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

word_to_ix = {'hello':0, 'world':1}
embeds = nn.Embedding(2,9)
hello_idx = torch.LongTensor([word_to_ix['hello']])
hello_idx = Variable(hello_idx)
hello_embed = embeds(hello_idx)
print(hello_embed)


tensor([[-0.0448, -1.9270,  2.6078,  0.7212,  0.9924, -0.1768,  0.2435,  0.0551,
          0.5441]], grad_fn=<EmbeddingBackward0>)


In [25]:
hello_idx

tensor([0])

In [56]:
class BiRNN(nn.Module):
    def __init__(self, vocab_size, embed_size, num_hiddens,
                 num_layers, **kwargs):
        super(BiRNN, self).__init__(**kwargs)
        self.embedding = nn.Embedding(vocab_size, embed_size)
        # 将bidirectional设置为True以获取双向循环神经网络
        self.encoder = nn.LSTM(embed_size, num_hiddens, num_layers=num_layers,
                                bidirectional=True)
        self.decoder = nn.Linear(4 * num_hiddens, 2)

    def forward(self, inputs):
        print(inputs.shape)
        # inputs的形状是（批量大小，时间步数）
        # 因为长短期记忆网络要求其输入的第一个维度是时间维，
        # 所以在获得词元表示之前，输入会被转置。
        # 输出形状为（时间步数，批量大小，词向量维度）
        embeddings = self.embedding(inputs.T)
        self.encoder.flatten_parameters()
        # 返回上一个隐藏层在不同时间步的隐状态，
        # outputs的形状是（时间步数，批量大小，2*隐藏单元数）
        outputs, _ = self.encoder(embeddings)
        # 连结初始和最终时间步的隐状态，作为全连接层的输入，
        # 其形状为（批量大小，4*隐藏单元数）
        encoding = torch.cat((outputs[0], outputs[-1]), dim=1)
        outs = self.decoder(encoding)
        return outs

In [57]:
embed_size, num_hiddens, num_layers = 100, 100, 2
net = BiRNN(len(vocab), embed_size, num_hiddens, num_layers)

In [58]:
net(aa[0])

torch.Size([64, 500])


tensor([[-0.0423, -0.0328],
        [-0.0410, -0.0294],
        [-0.0392, -0.0275],
        [-0.0697, -0.0273],
        [-0.0282, -0.0066],
        [-0.0265, -0.0172],
        [-0.0423, -0.0269],
        [-0.0610, -0.0153],
        [-0.0536, -0.0312],
        [-0.0420, -0.0221],
        [-0.0356, -0.0018],
        [-0.0580, -0.0231],
        [-0.0505, -0.0135],
        [-0.0599, -0.0189],
        [-0.0392, -0.0199],
        [-0.0256, -0.0212],
        [-0.0511, -0.0156],
        [-0.0517, -0.0301],
        [-0.0544, -0.0334],
        [-0.0569, -0.0335],
        [-0.0509, -0.0413],
        [-0.0498, -0.0198],
        [-0.0630, -0.0352],
        [-0.0405, -0.0234],
        [-0.0548, -0.0194],
        [-0.0537, -0.0202],
        [-0.0474, -0.0226],
        [-0.0350, -0.0149],
        [-0.0497, -0.0399],
        [-0.0538, -0.0291],
        [-0.0294, -0.0077],
        [-0.0438, -0.0368],
        [-0.0059, -0.0299],
        [-0.0809, -0.0205],
        [-0.0223, -0.0085],
        [-0.0438, -0

In [60]:
aa[0][0]

tensor([   10,    38,   181,     5,    26,    31,     4,   150,   135,   110,
            1,  1920,     6,  2417,   298,   145,    69,     1,    20,     0,
           89,   206,     1,  1920,     7, 28104,    11,   544,     0,   103,
          140,    24,   207,   431,     5,    66,    11,    17,    85,   102,
          158,     0,  2478,  1579,  3331,   189,  9585,    12,     0,    40,
          148,     8,    13,    39,  1095,    91, 28318,  1403,   140,   636,
           11,   540,    26,    32,   916,  4483,    17,   131,    57,  1983,
            9,    11,    49,    26,     2,   310,     0,    23,     7,     1,
            0, 26858,  1855,    11,    13,  1360,    59,     0,    12,     0,
           15,    36,     1,   309,  1458,     3,  2886,   287,     3,     1,
          138,    13,    39,    98,    11,    13,   171, 40872,    12,    92,
          563,   165,    64,  3400,     3,   173,     0,    12,  9701,   344,
           53,  2760,    83,     0,     3,     8,   262,     8, 

In [61]:
len(aa[0][0])

500